<a href="https://colab.research.google.com/github/edsonportosilva/OptiCommPy/blob/main/examples/test_WDM_transmission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simulation of coherent WDM transmission

In [1]:
if 'google.colab' in str(get_ipython()):    
    ! git clone -b main https://github.com/edsonportosilva/OptiCommPy
    from os import chdir as cd
    cd('/content/OptiCommPy/')
    ! pip install . 

In [57]:
import matplotlib.pyplot as plt
import numpy as np

from optic.dsp import pulseShape, firFilter, decimate, symbolSync, pnorm
from optic.models import phaseNoise, pdmCoherentReceiver

try:
    from optic.modelsGPU import manakovSSF, manakovDBP
except:
    from optic.models import manakovSSF

from optic.tx import simpleWDMTx
from optic.core import parameters
from optic.equalization import edc, mimoAdaptEqualizer
from optic.carrierRecovery import cpr
from optic.metrics import fastBERcalc, monteCarloGMI, monteCarloMI, signal_power, calcEVM
from optic.plot import pconst, plotPSD

import scipy.constants as const

import logging as logg
logg.basicConfig(level=logg.INFO, format='%(message)s', force=True)

from copy import deepcopy

In [3]:
from IPython.core.display import HTML
from IPython.core.pylabtools import figsize

HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
""")

In [4]:
figsize(10, 3)

In [5]:
%load_ext autoreload
%autoreload 2
#%load_ext line_profiler


## Transmitter

**Polarization multiplexed WDM signal generation**

In [42]:
# Transmitter parameters:
paramTx = parameters()
paramTx.M   = 64           # order of the modulation format
paramTx.Rs  = 32e9         # symbol rate [baud]
paramTx.SpS = 16           # samples per symbol
paramTx.pulse = 'rrc'      # pulse shaping filter
paramTx.Ntaps = 2*4096     # number of pulse shaping filter coefficients
paramTx.alphaRRC = 0.01    # RRC rolloff
paramTx.Pch_dBm = 0        # power per WDM channel [dBm]
paramTx.Nch     = 11       # number of WDM channels
paramTx.Fc      = 193.1e12 # central optical frequency of the WDM spectrum
paramTx.lw      = 100e3    # laser linewidth in Hz
paramTx.freqSpac = 37.5e9  # WDM grid spacing
paramTx.Nmodes = 2         # number of signal modes [2 for polarization multiplexed signals]
paramTx.Nbits = int(np.log2(paramTx.M)*1e5) # total number of bits per polarization

# generate WDM signal
sigWDM_Tx, symbTx_, paramTx = simpleWDMTx(paramTx)

  0%|          | 0/11 [00:00<?, ?it/s]

channel 0	 fc : 192.9125 THz
  mode #0	 power: -3.01 dBm
  mode #1	 power: -3.01 dBm
channel 0	 power: -0.00 dBm

channel 1	 fc : 192.9500 THz
  mode #0	 power: -3.01 dBm
  mode #1	 power: -3.01 dBm
channel 1	 power: -0.00 dBm

channel 2	 fc : 192.9875 THz
  mode #0	 power: -3.01 dBm
  mode #1	 power: -3.01 dBm
channel 2	 power: 0.00 dBm

channel 3	 fc : 193.0250 THz
  mode #0	 power: -3.01 dBm
  mode #1	 power: -3.01 dBm
channel 3	 power: -0.00 dBm

channel 4	 fc : 193.0625 THz
  mode #0	 power: -3.01 dBm
  mode #1	 power: -3.01 dBm
channel 4	 power: 0.00 dBm

channel 5	 fc : 193.1000 THz
  mode #0	 power: -3.01 dBm
  mode #1	 power: -3.01 dBm
channel 5	 power: -0.00 dBm

channel 6	 fc : 193.1375 THz
  mode #0	 power: -3.01 dBm
  mode #1	 power: -3.01 dBm
channel 6	 power: 0.00 dBm

channel 7	 fc : 193.1750 THz
  mode #0	 power: -3.01 dBm
  mode #1	 power: -3.01 dBm
channel 7	 power: -0.00 dBm

channel 8	 fc : 193.2125 THz
  mode #0	 power: -3.01 dBm
  mode #1	 power: -3.01 dBm
channe

**Nonlinear fiber propagation with the split-step Fourier method**

In [ ]:
# optical channel parameters
paramCh = parameters()
paramCh.Ltotal = 700     # total link distance [km]
paramCh.Lspan  = 50      # span length [km]
paramCh.alpha = 0.2      # fiber loss parameter [dB/km]
paramCh.D = 16           # fiber dispersion parameter [ps/nm/km]
paramCh.gamma = 1.3      # fiber nonlinear parameter [1/(W.km)]
paramCh.Fc = paramTx.Fc  # central optical frequency of the WDM spectrum
paramCh.hz = 0.5         # step-size of the split-step Fourier method [km]
paramCh.maxIter = 5      # maximum number of convergence iterations per step
paramCh.tol = 1e-5       # error tolerance per step
paramCh.nlprMethod = True # use adaptive step-size based o maximum nonlinear phase-shift?
paramCh.maxNlinPhaseRot = 2e-2 # maximum nonlinear phase-shift per step
paramCh.prgsBar = True   # show progress bar?
#paramCh.saveSpanN = [1, 5, 9, 14]
Fs = paramTx.Rs*paramTx.SpS # sampling rate

# DBP parameters
paramDBP = deepcopy(paramCh)
paramDBP.nlprMethod = False
paramDBP.hz = 10
runDBP = True

### Receiver parameters

Fc = paramCh.Fc
Ts = 1/Fs
freqGrid = paramTx.freqGrid
    
## LO parameters
FO      = 150e6                 # frequency offset
lw      = 100e3                 # linewidth
Plo_dBm = 10                    # power in dBm
Plo     = 10**(Plo_dBm/10)*1e-3 # power in W
ϕ_lo    = 0                     # initial phase in rad    

## photodiodes parameters
paramPD = parameters()
paramPD.B = paramTx.Rs
paramPD.Fs = Fs    
paramPD.ideal = True
    
Powers = paramTx.Pch_dBm + np.arange(-5,2,0.5)
scale = np.arange(-5,2,0.5)

BER = np.zeros((2,len(Powers)))
SER = np.zeros((2,len(Powers)))
MI  = np.zeros((2,len(Powers)))
GMI = np.zeros((2,len(Powers)))
NGMI = np.zeros((2,len(Powers)))
SNR = np.zeros((2,len(Powers)))
EVM = np.zeros((2,len(Powers)))

for indP, G in enumerate(scale):
    # nonlinear signal propagation
    G_lin = 10**(G/10)

    sigWDM, paramCh = manakovSSF(np.sqrt(G_lin)*sigWDM_Tx, Fs, paramCh)
    print('sigWDM power: ', round(10*np.log10(signal_power(sigWDM)/paramTx.Nch /1e-3),2),'dBm')
    
    ### WDM channels coherent detection and demodulation

    ### Receiver

    # parameters
    chIndex  = int(np.floor(paramTx.Nch/2))      # index of the channel to be demodulated

#     print('Demodulating channel #%d , fc: %.4f THz, λ: %.4f nm\n'\
#           %(chIndex, (Fc + freqGrid[chIndex])/1e12, const.c/(Fc + freqGrid[chIndex])/1e-9))

    symbTx = symbTx_[:,:,chIndex]

    #  set local oscillator (LO) parameters:   
    Δf_lo   = freqGrid[chIndex]+FO  # downshift of the channel to be demodulated

    # generate LO field
    π       = np.pi
    t       = np.arange(0, len(sigWDM))*Ts
    ϕ_pn_lo = phaseNoise(lw, len(sigWDM), Ts)
    sigLO   = np.sqrt(Plo)*np.exp(1j*(2*π*Δf_lo*t + ϕ_lo + ϕ_pn_lo))

    #### polarization multiplexed coherent optical receiver
    θsig = π/3 # polarization rotation angle
    sigRx = pdmCoherentReceiver(sigWDM, sigLO, θsig, paramPD)

    ### Matched filtering and CD compensation

    # Rx filtering
    
    # Matched filtering
    if paramTx.pulse == 'nrz':
        pulse = pulseShape('nrz', paramTx.SpS)
    elif paramTx.pulse == 'rrc':
        pulse = pulseShape('rrc', paramTx.SpS, N=paramTx.Ntaps, alpha=paramTx.alphaRRC, Ts=1/paramTx.Rs)

    pulse = pnorm(pulse)
    sigRx = firFilter(pulse, sigRx)

    # CD compensation/digital backpropagation
    if runDBP:
        Pch = 10**((G + paramTx.Pch_dBm)/10)*1e-3
        sigRx = np.sqrt(Pch/2)*pnorm(sigRx)
        print('channel input power (DBP): ', round(10*np.log10(signal_power(sigRx)/1e-3),3),'dBm')

        sigRx,_ = manakovDBP(sigRx, Fs, paramDBP)    
    else:
        sigRx = edc(sigRx, paramCh.Ltotal, paramCh.D, Fc-Δf_lo, Fs)

    ### Downsampling to 2 samples/symbol and re-synchronization with transmitted sequences

    # decimation
    paramDec = parameters()
    paramDec.SpS_in  = paramTx.SpS
    paramDec.SpS_out = 2
    sigRx = decimate(sigRx, paramDec)

    symbRx = symbolSync(sigRx, symbTx, 2)

    ### Power normalization

    x = pnorm(sigRx)
    d = pnorm(symbRx)

    ### Adaptive equalization

    # adaptive equalization parameters
    paramEq = parameters()
    paramEq.nTaps = 15
    paramEq.SpS = paramDec.SpS_out
    paramEq.numIter = 5
    paramEq.storeCoeff = False
    paramEq.M = paramTx.M
    paramEq.L = [int(0.2*d.shape[0]), int(0.8*d.shape[0])]
    paramEq.prgsBar = False

    if paramTx.M == 4:
        paramEq.alg = ['cma','cma'] # QPSK
        paramEq.mu = [5e-3, 1e-3] 
    else:
        paramEq.alg = ['da-rde','rde'] # M-QAM
        paramEq.mu = [5e-3, 2e-4] 

    y_EQ, H, errSq, Hiter = mimoAdaptEqualizer(x, dx=d, paramEq=paramEq)

    ### Carrier phase recovery

    paramCPR = parameters()
    paramCPR.alg = 'bps'
    paramCPR.M   = paramTx.M
    paramCPR.N   = 75
    paramCPR.B   = 64
    paramCPR.pilotInd = np.arange(0, len(y_EQ), 20) 

    y_CPR, θ = cpr(y_EQ, symbTx=d, paramCPR=paramCPR)

    y_CPR = pnorm(y_CPR)

    discard = 5000

    ### Evaluate transmission metrics

    ind = np.arange(discard, d.shape[0]-discard)

    # remove phase and polarization ambiguities for QPSK signals
    if paramTx.M == 4:   
        d = symbTx
        # find rotations after CPR and/or polarizations swaps possibly added at the output the adaptive equalizer:
        rot0 = [np.mean(pnorm(symbTx[ind,0])/pnorm(y_CPR[ind,0])), np.mean(pnorm(symbTx[ind,1])/pnorm(y_CPR[ind,0]))]
        rot1 = [np.mean(pnorm(symbTx[ind,1])/pnorm(y_CPR[ind,1])), np.mean(pnorm(symbTx[ind,0])/pnorm(y_CPR[ind,1]))]

        if np.argmax(np.abs(rot0)) == 1 and np.argmax(np.abs(rot1)) == 1:      
            y_CPR_ = y_CPR.copy() 
            # undo swap and rotation 
            y_CPR[:,0] = pnorm(rot1[np.argmax(np.abs(rot1))]*y_CPR_[:,1]) 
            y_CPR[:,1] = pnorm(rot0[np.argmax(np.abs(rot0))]*y_CPR_[:,0])
        else:
            # undo rotation
            y_CPR[:,0] = pnorm(rot0[np.argmax(np.abs(rot0))]*y_CPR[:,0])
            y_CPR[:,1] = pnorm(rot1[np.argmax(np.abs(rot1))]*y_CPR[:,1])


    BER[:,indP], SER[:,indP], SNR[:,indP] = fastBERcalc(y_CPR[ind,:], d[ind,:], paramTx.M, 'qam')
    GMI[:,indP], NGMI[:,indP] = monteCarloGMI(y_CPR[ind,:], d[ind,:], paramTx.M, 'qam')
    MI[:,indP] = monteCarloMI(y_CPR[ind,:], d[ind,:], paramTx.M, 'qam')
    EVM[:,indP] = calcEVM(y_CPR[ind,:], paramTx.M, 'qam', d[ind,:])

    print('      pol.X      pol.Y      ')
    print(' SER: %.2e,  %.2e'%(SER[0,indP], SER[1,indP]))
    print(' BER: %.2e,  %.2e'%(BER[0,indP], BER[1,indP]))
    print(' SNR: %.2f dB,  %.2f dB'%(SNR[0,indP], SNR[1,indP]))
    print(' EVM: %.2f %%,    %.2f %%'%(EVM[0,indP]*100, EVM[1,indP]*100))
    print('  MI: %.2f bits, %.2f bits'%(MI[0,indP], MI[1,indP]))
    print(' GMI: %.2f bits, %.2f bits'%(GMI[0,indP], GMI[1,indP]))
    print('NGMI: %.2f,      %.2f'%(NGMI[0,indP], NGMI[1,indP]))

  0%|          | 0/14 [00:00<?, ?it/s]

sigWDM power:  -4.97 dBm
channel input power (DBP):  -5.0 dBm


  0%|          | 0/14 [00:00<?, ?it/s]

da-rde - training stage #0
da-rde pre-convergence training iteration #0
da-rde MSE = 0.020299.
da-rde pre-convergence training iteration #1
da-rde MSE = 0.018172.
da-rde pre-convergence training iteration #2
da-rde MSE = 0.018123.
da-rde pre-convergence training iteration #3
da-rde MSE = 0.018089.
da-rde pre-convergence training iteration #4
da-rde MSE = 0.018064.
rde - training stage #1
rde MSE = 0.007341.


      pol.X      pol.Y      
 SER: 1.90e-02,  1.81e-02
 BER: 3.19e-03,  3.02e-03
 SNR: 21.47 dB,  21.53 dB
 EVM: 0.71 %,    0.70 %
  MI: 5.92 bits, 5.92 bits
 GMI: 5.92 bits, 5.93 bits
NGMI: 0.99,      0.99


...

sigWDM power:  -0.99 dBm
channel input power (DBP):  -1.0 dBm


  0%|          | 0/14 [00:00<?, ?it/s]

da-rde - training stage #0
da-rde pre-convergence training iteration #0
da-rde MSE = 0.023607.
da-rde pre-convergence training iteration #1
da-rde MSE = 0.013736.
da-rde pre-convergence training iteration #2
da-rde MSE = 0.013620.
da-rde pre-convergence training iteration #3
da-rde MSE = 0.013545.
da-rde pre-convergence training iteration #4
da-rde MSE = 0.013493.
rde - training stage #1
rde MSE = 0.006692.


      pol.X      pol.Y      
 SER: 9.33e-03,  9.17e-03
 BER: 1.56e-03,  1.53e-03
 SNR: 22.36 dB,  22.34 dB
 EVM: 0.58 %,    0.58 %
  MI: 5.96 bits, 5.96 bits
 GMI: 5.96 bits, 5.96 bits
NGMI: 0.99,      0.99


  0%|          | 0/14 [00:00<?, ?it/s]